In [ ]:
!pip install pytesseract pdf2image fitz spacy scispacy rapidfuzz
!python -m spacy download en_core_web_sm
!pip install mistralai
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev
!apt-get install poppler-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 129.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest

In [ ]:
!pip install spacy==3.4.1
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz

  Using cached spacy-3.4.1.tar.gz (1.2 MB)
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz (120.2 MB)
  Preparing metadata (setup.py) ... done
  Using cached spacy-3.4.4.tar.gz (1.2 MB)
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is 

In [ ]:
import os
from google.colab import userdata

# Retrieve API key from Colab's secret storage
MISTRAL_API_TOKEN = userdata.get('Mistral_API')

# Store it in environment variables for secure access
os.environ["MISTRAL_API_KEY"] = MISTRAL_API_TOKEN


In [ ]:
# Upload, OCR, parse CBC table, and compare to reference ranges (Colab-ready)
# Requirements: tesseract & pytesseract already installed in your Colab env.
from google.colab import files
import cv2, numpy as np, pytesseract, re, io, os
from PIL import Image

# 1) Upload your image
print("Upload the CBC image (choose the file).")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")
file_name = list(uploaded.keys())[0]
print("Uploaded:", file_name)

# 2) Preprocess image for better OCR
img = cv2.imdecode(np.frombuffer(uploaded[file_name], np.uint8), cv2.IMREAD_COLOR)
# Resize to increase OCR accuracy
scale = 2.0
img = cv2.resize(img, (0,0), fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Denoise and increase contrast
gray = cv2.bilateralFilter(gray, 9, 75, 75)
# Apply adaptive thresholding to binarize
th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                           cv2.THRESH_BINARY, 25, 10)

# Save a copy to inspect if needed
cv2.imwrite("/content/_ocr_preprocessed.png", th)
print("Preprocessed image saved to /content/_ocr_preprocessed.png (optional).")

# 3) Run pytesseract OCR
custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789.%/abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ-() '
text = pytesseract.image_to_string(th, config=custom_config)
print("\n--- OCR TEXT (first 500 chars) ---\n")
print(text[:500])
print("\n--- End OCR preview ---\n")

# 4) Define expected keys and simple matching rules
expected_keys = {
    "white": "WBC",
    "white blood": "WBC",
    "wbc": "WBC",
    "red blood": "RBC",
    "rbc": "RBC",
    "hemoglobin": "Hemoglobin",
    "hb": "Hemoglobin",
    "hematocrit": "Hematocrit",
    "mcv": "MCV",
    "mch": "MCH",
    "mchc": "MCHC",
    "rdw": "RDW",
    "platelet": "Platelet Count",
    "platelets": "Platelet Count",
    "neutrophil": "Neutrophil %",
    "lymphocyte": "Lymphocyte %",
    "lymphocytes": "Lymphocyte %",
    "monocyte": "Monocyte %",
    "eosinophil": "Eosinophil %",
    "basophil": "Basophil %",
    "abs. neutrophil": "Abs. Neutrophil",
    "abs neutrophil": "Abs. Neutrophil",
    "abs. lymphocyte": "Abs. Lymphocyte",
    "abs lymphocyte": "Abs. Lymphocyte",
    "abs. monocyte": "Abs. Monocyte",
    "abs monocyte": "Abs. Monocyte",
    "abs. eosinophil": "Abs. Eosinophil",
    "abs eosinophil": "Abs. Eosinophil",
    "abs. basophil": "Abs. Basophil",
    "abs basophil": "Abs. Basophil"
}

# 5) Parse lines and extract first numeric value in line for matched key
lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
extracted = {}

num_re = re.compile(r'(-?\d+(?:\.\d+)?)')

for ln in lines:
    low_ln = ln.lower()
    for kfrag, canonical in expected_keys.items():
        if kfrag in low_ln:
            # find all numbers in line
            nums = num_re.findall(ln)
            if nums:
                # choose the first number that looks like a result (not a range like 4.0-11.0)
                # prefer single numbers; if first looks like "4.0 - 11.0" it still captures 4.0 and 11.0
                # strategy: if line contains '-' with spaces or '/', try to pick the one outside ranges by context:
                # but simplest: use first numeric token that is not part of 'standard range' keywords
                val = nums[0]
                extracted[canonical] = val
            else:
                # no numeric found on that line; leave it
                pass

# 6) Add manual fixes if OCR missed percent signs/units or split text incorrectly
# (Optional) If Platelet Count not found try look for '149' pattern specifically
if "Platelet Count" not in extracted:
    # try to find "150" vicinity numbers highlighted in OCR text lines
    for ln in lines:
        if "plate" in ln.lower():
            m = num_re.search(ln)
            if m:
                extracted["Platelet Count"] = m.group(1)

# 7) Reference ranges used in the image (as floats where applicable)
STANDARD_RANGES = {
    "WBC": (4.0, 11.0, "K/µL"),
    "RBC": (4.40, 6.00, "M/µL"),
    "Hemoglobin": (13.5, 18.0, "g/dL"),
    "Hematocrit": (40.0, 52.0, "%"),
    "MCV": (80.0, 100.0, "fL"),
    "MCH": (27.0, 33.0, "pg"),
    "MCHC": (31.0, 36.0, "g/dL"),
    "RDW": (0.0, 16.4, "%"),   # image shows <16.4
    "Platelet Count": (150.0, 400.0, "K/µL"),
    "Neutrophil %": (49.0, 74.0, "%"),
    "Lymphocyte %": (26.0, 46.0, "%"),
    "Monocyte %": (2.0, 12.0, "%"),
    "Eosinophil %": (0.0, 5.0, "%"),
    "Basophil %": (0.0, 2.0, "%"),
    "Abs. Neutrophil": (2.0, 8.0, "K/µL"),
    "Abs. Lymphocyte": (1.0, 5.1, "K/µL"),
    "Abs. Monocyte": (0.0, 0.8, "K/µL"),
    "Abs. Eosinophil": (0.0, 0.5, "K/µL"),
    "Abs. Basophil": (0.0, 0.2, "K/µL")
}

# 8) Compare and print results
def safe_float(x):
    try:
        return float(x)
    except:
        return None

comparison = {}
for param, val in extracted.items():
    numeric = safe_float(val)
    if numeric is None:
        comparison[param] = ("Invalid", val)
    elif param in STANDARD_RANGES:
        low, high, unit = STANDARD_RANGES[param]
        if numeric < low:
            comparison[param] = ("Low", f"{numeric} {unit}")
        elif numeric > high:
            comparison[param] = ("High", f"{numeric} {unit}")
        else:
            comparison[param] = ("Normal", f"{numeric} {unit}")
    else:
        comparison[param] = ("No reference", val)

print("\n--- Extracted values ---")
for k,v in extracted.items():
    print(f"{k}: {v}")

print("\n--- Comparison vs standard ranges ---")
for k,(status,info) in comparison.items():
    print(f"{k}: {status} ({info})")

# 9) If OCR missed some expected keys, print a warning and show OCR text (so you can correct manually)
missing = [k for k in STANDARD_RANGES.keys() if k not in extracted]
if missing:
    print("\nNote: OCR did not extract these expected parameters automatically:", missing)
    print("You can inspect /content/_ocr_preprocessed.png and the OCR text above, or add manual values as needed.")


Upload the CBC image (choose the file).


Saving medical-transcription-grammar-how-to-transcribe-cbc-and-cmp (1).jpg to medical-transcription-grammar-how-to-transcribe-cbc-and-cmp (1) (1).jpg
Uploaded: medical-transcription-grammar-how-to-transcribe-cbc-and-cmp (1) (1).jpg
Preprocessed image saved to /content/_ocr_preprocessed.png (optional).

--- OCR TEXT (first 500 chars) ---

Component YourValueStandardRange UnitsFlag
WhiteBloodCellCount 5.4 4.0-11.0 K/uL
RedBloodCellCount 5.20 4.40-6.00 M/uL
Hemoglobin 16.0 13.5-18.0 g/dL
Hematocrit 47.2 40.0-52.0 %

MCV 91 80-100 fL

MCH 30.8 27.0-33.0 pg

MCHC 33.9 31.0-36.0 g/dL
RDW 12.7 16.4- %
PlateletCount 149 1S50-400 K/uL L
DifferentialType Automated

Neutrophil% 56 49.0-74.0 %
Lymphocyte% 23 26.0-46.0 % L
Monocyte% 13 2.0-12.0 % H
Eosinophil% 7 0.0-S.0 % H
Basophil% 1 0.0-2.0 %
Abs.Neutrophil 3.1 2.0-8.0 K/uL
Abs.Lymphocyte

--- End OCR preview ---


--- Extracted values ---
WBC: 5.4
Hemoglobin: 16.0
Hematocrit: 47.2
MCV: 91
MCH: 33.9
MCHC: 33.9
RDW: 12.7
Platelet Count: 149
Neutr

In [ ]:
!pip install pandas scikit-learn numpy

In [ ]:
# Upload a ZIP file and extract it (Colab-friendly)
from google.colab import files
import zipfile, os
import pandas as pd

print("Please choose the ZIP file to upload (e.g. archive (11).zip).")
uploaded = files.upload()   # prompts file picker

if not uploaded:
    raise SystemExit("No file uploaded. Re-run the cell and choose a file.")

# Use the first uploaded file
zip_filename = list(uploaded.keys())[0]
zip_path = os.path.join("/content", zip_filename)

# Ensure the uploaded bytes are saved to /content
with open(zip_path, "wb") as f:
    f.write(uploaded[zip_filename])

extract_path = "/content/extracted_dataset"
os.makedirs(extract_path, exist_ok=True)

# Extract the ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_path)

print(f"✅ Uploaded and extracted: {zip_filename} -> {extract_path}")
print("Extracted files:")
print(os.listdir(extract_path))

Please choose the ZIP file to upload (e.g. archive (11).zip).


Saving archive (11).zip to archive (11) (2).zip
✅ Uploaded and extracted: archive (11) (2).zip -> /content/extracted_dataset
Extracted files:
['nasariancad.csv']


In [ ]:
import os

# List all files inside the extracted dataset folder
files = os.listdir(extract_path)
print("Extracted Files:", files)


Extracted Files: ['nasariancad.csv']


In [ ]:
# Identify CSV file
csv_file = [f for f in files if f.endswith(".csv")]
if not csv_file:
    raise FileNotFoundError("No CSV file found in the ZIP archive!")
csv_file_path = os.path.join(extract_path, csv_file[0])
print(f"✅ CSV File Found: {csv_file[0]}")

# Load dataset
df = pd.read_csv(csv_file_path)

# Display first few rows
print("✅ Dataset Loaded Successfully!")
df.head()

✅ CSV File Found: nasariancad.csv
✅ Dataset Loaded Successfully!


,heartattack,Age,Weight,Length,BMI,DM,HTN,FAMILYHTN,CurrentSmoker,EXSmoker,...,BUN,RBC,HB,POLY,WBC,Lymph,eo,PLT,HTC,angiographyCAD
0,1,59,75,177,23.93,2,1,2,1,2,...,13.2,4.12,11.9,51,7700,42,0,287,36.0,2
1,1,48,82,185,27.39,1,1,1,1,2,...,14.2,3.88,16.1,59,7500,40,1,244,41.0,2
2,1,51,95,174,31.02,1,1,1,1,1,...,13.4,5.04,12.5,60,6500,44,0,325,40.0,1
3,1,55,70,172,24.80,1,1,1,1,1,...,11.5,4.16,13.0,57,6000,45,1,203,39.0,2
4,1,51,104,167,37.29,2,2,1,1,1,...,16.4,4.40,13.3,50,6000,52,1,237,40.0,1


In [ ]:
# Check missing values
print("Missing Values Before Handling:\n", df.isnull().sum())

# Fill numerical columns with mean
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    df[col].fillna(df[col].mean(), inplace=True)

# Fill categorical columns with mode
for col in df.select_dtypes(include=['object']).columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

print("✅ Missing Values Handled Successfully!")
df.isnull().sum()


Missing Values Before Handling:
 heartattack            0
Age                    0
Weight                 0
Length                 0
BMI                    0
DM                     0
HTN                    0
FAMILYHTN              0
CurrentSmoker          0
EXSmoker               0
FH                     0
Obesity                0
CHAGHISHEKAMI          0
CRF                    0
CVA                    0
Airwaydisease          0
ThyroidDisease         0
HLP                    0
STRESS                 0
noise                  0
shiftwork              0
BP                     0
PR                     0
Edema                  0
WeakPeripheralPulse    0
Lungrales              0
SystolicMurmur         0
DiastolicMurmur        0
 ChestPain             0
Dyspnea                0
heartbeat              0
syanoz                 0
Function\n             0
exercisetest           0
arytmi                 0
FBS                    0
CR                     0
TG                     0
LDL              

/tmp/ipython-input-2780377838.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


,0
heartattack,0
Age,0
Weight,0
Length,0
BMI,0
DM,0
HTN,0
FAMILYHTN,0
CurrentSmoker,0
EXSmoker,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define Features (X) and Target (y)
X = df.drop(columns=["heartattack"])  # Drop target column
y = df["heartattack"]  # Target variable

# Convert categorical values to numerical (if any)
X = pd.get_dummies(X)  # Convert categorical columns into one-hot encoding

# Normalize numerical features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset into Training (80%) and Testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("\n✅ Data Splitting Completed!")
print(f"📊 Training Set: {X_train.shape}, Testing Set: {X_test.shape}")


✅ Data Splitting Completed!
📊 Training Set: (120, 49), Testing Set: (30, 49)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Model
print("\n🚀 Training the Model...")
model.fit(X_train, y_train)

print("✅ Model Training Completed!")


🚀 Training the Model...
✅ Model Training Completed!


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Make Predictions
y_pred = model.predict(X_test)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\n🎯 Model Accuracy: {accuracy:.2f}")

# Display Classification Report
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred))



🎯 Model Accuracy: 0.90

📊 Classification Report:
              precision    recall  f1-score   support

           1       0.90      1.00      0.95        27
           2       0.00      0.00      0.00         3

    accuracy                           0.90        30
   macro avg       0.45      0.50      0.47        30
weighted avg       0.81      0.90      0.85        30



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pickle

# Save the trained model
model_filename = "/content/trained_model.pkl"
with open(model_filename, "wb") as file:
    pickle.dump(model, file)

print(f"\n✅ Model Saved Successfully as: {model_filename}")


✅ Model Saved Successfully as: /content/trained_model.pkl


In [ ]:
# Manually define extracted_values (use the numbers from your image or edit them)
extracted_values = {
    "WBC": "5.4",
    "RBC": "5.20",
    "Hemoglobin": "16.0",
    "Hematocrit": "47.2",
    "MCV": "91",
    "MCH": "30.8",
    "MCHC": "33.9",
    "RDW": "12.7",
    "Platelet Count": "149",
    "Neutrophil %": "56",
    "Lymphocyte %": "23",
    "Monocyte %": "13",
    "Eosinophil %": "7",
    "Basophil %": "1",
    "Abs. Neutrophil": "3.1",
    "Abs. Lymphocyte": "1.2",
    "Abs. Monocyte": "0.7",
    "Abs. Eosinophil": "0.4",
    "Abs. Basophil": "0.0"
}
print("extracted_values defined. Keys:", list(extracted_values.keys()))

extracted_values defined. Keys: ['WBC', 'RBC', 'Hemoglobin', 'Hematocrit', 'MCV', 'MCH', 'MCHC', 'RDW', 'Platelet Count', 'Neutrophil %', 'Lymphocyte %', 'Monocyte %', 'Eosinophil %', 'Basophil %', 'Abs. Neutrophil', 'Abs. Lymphocyte', 'Abs. Monocyte', 'Abs. Eosinophil', 'Abs. Basophil']


In [ ]:
# Quick check
import json
if 'extracted_values' in globals():
    print("extracted_values present. Preview:")
    print(json.dumps(extracted_values, indent=2))
else:
    print("extracted_values NOT found. Re-run upload or manual define cell.")

extracted_values present. Preview:
{
  "WBC": "5.4",
  "RBC": "5.20",
  "Hemoglobin": "16.0",
  "Hematocrit": "47.2",
  "MCV": "91",
  "MCH": "30.8",
  "MCHC": "33.9",
  "RDW": "12.7",
  "Platelet Count": "149",
  "Neutrophil %": "56",
  "Lymphocyte %": "23",
  "Monocyte %": "13",
  "Eosinophil %": "7",
  "Basophil %": "1",
  "Abs. Neutrophil": "3.1",
  "Abs. Lymphocyte": "1.2",
  "Abs. Monocyte": "0.7",
  "Abs. Eosinophil": "0.4",
  "Abs. Basophil": "0.0"
}


In [ ]:
import numpy as np
import pandas as pd

# Define the required feature columns from the dataset
required_features = [
    "Age", "Weight", "Length", "BMI", "DM", "HTN", "FAMILYHTN", "CurrentSmoker", "EXSmoker", "FH",
    "Obesity", "CHAGHISHEKAMI", "CRF", "CVA", "Airwaydisease", "ThyroidDisease", "HLP", "STRESS", "noise",
    "shiftwork", "BP_Systolic", "BP_Diastolic", "Heart Rate", "Edema", "WeakPeripheralPulse", "Lungrales",
    "SystolicMurmur", "DiastolicMurmur", "ChestPain", "Dyspnea", "heartbeat", "syanoz", "Function",
    "exercisetest", "arytmi", "FBS", "CR", "TG", "LDL", "HDL", "BUN", "RBC", "Hemoglobin", "POLY",
    "WBC", "Lymph", "eo", "Platelets", "Hematocrit"
]

# Extracted values from PDF (Example: Dynamically obtained)
pdf_values = extracted_values

# Convert BP to Systolic & Diastolic if available
if "Blood Pressure" in pdf_values:
    systolic_bp, diastolic_bp = map(int, pdf_values["Blood Pressure"].split('/'))
    pdf_values["BP_Systolic"] = systolic_bp
    pdf_values["BP_Diastolic"] = diastolic_bp

# ✅ Dynamically Create Input Array with Missing Values as 0
new_input= {feature: pdf_values.get(feature, 0) for feature in required_features}

print(new_input)

# Convert new input to DataFrame with feature names
columns = df.drop(columns=["heartattack"]).columns  # Assuming 'heartattack' is the target column
new_input_df = pd.DataFrame([new_input], columns=columns)

# Scale the new input
new_input_scaled = scaler.transform(new_input_df)

# Load the trained model
with open("/content/trained_model.pkl", "rb") as file:
    trained_model = pickle.load(file)

# Make prediction
prediction = trained_model.predict(new_input_scaled)

# Print Prediction Result
print("\n🔍 Prediction Result:")
print("✅ Heart Attack Risk: Yes" if prediction[0] == 1 else "❌ No Heart Attack Risk")


{'Age': 0, 'Weight': 0, 'Length': 0, 'BMI': 0, 'DM': 0, 'HTN': 0, 'FAMILYHTN': 0, 'CurrentSmoker': 0, 'EXSmoker': 0, 'FH': 0, 'Obesity': 0, 'CHAGHISHEKAMI': 0, 'CRF': 0, 'CVA': 0, 'Airwaydisease': 0, 'ThyroidDisease': 0, 'HLP': 0, 'STRESS': 0, 'noise': 0, 'shiftwork': 0, 'BP_Systolic': 0, 'BP_Diastolic': 0, 'Heart Rate': 0, 'Edema': 0, 'WeakPeripheralPulse': 0, 'Lungrales': 0, 'SystolicMurmur': 0, 'DiastolicMurmur': 0, 'ChestPain': 0, 'Dyspnea': 0, 'heartbeat': 0, 'syanoz': 0, 'Function': 0, 'exercisetest': 0, 'arytmi': 0, 'FBS': 0, 'CR': 0, 'TG': 0, 'LDL': 0, 'HDL': 0, 'BUN': 0, 'RBC': '5.20', 'Hemoglobin': '16.0', 'POLY': 0, 'WBC': '5.4', 'Lymph': 0, 'eo': 0, 'Platelets': 0, 'Hematocrit': '47.2'}

🔍 Prediction Result:
✅ Heart Attack Risk: Yes


In [ ]:
# Rewritten prediction cell (robust, follows your original logic)
import re
import numpy as np
import pandas as pd
import pickle
import os

# --- Helpers ---
def parse_number(val):
    """Try to convert a value to float. Return None if not possible.
    Keeps BP-like strings (e.g. '120/80') intact as str.
    """
    if val is None:
        return None
    if isinstance(val, (int, float)):
        return float(val)
    s = str(val).strip()
    if not s:
        return None
    # If looks like BP (contains '/'), return the raw string so we can split later
    if '/' in s:
        return s
    # remove commas, percent signs, and common non-numeric noise
    s = s.replace(',', '')
    s = s.replace('%', '')
    # keep digits, dot and minus
    s = re.sub(r'[^\d\.\-]', '', s)
    try:
        return float(s)
    except:
        return None

def extract_bp_parts(s):
    """Given a BP-like string, return (systolic, diastolic) as floats, or (None, None)."""
    if not isinstance(s, str):
        return (None, None)
    m = re.search(r'(\d{2,3})\s*[\/\-]\s*(\d{2,3})', s)
    if not m:
        return (None, None)
    try:
        return float(m.group(1)), float(m.group(2))
    except:
        return (None, None)

# --- Preconditions check ---
if 'extracted_values' not in globals():
    raise NameError(
        "Variable 'extracted_values' not found. Please produce it first (run OCR/parse cell) "
        "or set it manually. Example:\n"
        "extracted_values = {'WBC':'5.4','RBC':'5.20','Hemoglobin':'16.0', ...}"
    )

if 'df' not in globals():
    raise NameError("Training DataFrame 'df' not found. Run the dataset load/preprocess cells first (Cells 8/9).")

if 'scaler' not in globals():
    raise NameError("Feature scaler 'scaler' not found. Run the scaling cell (Cell 10) to create the StandardScaler instance.")

# Load model (prefer in-memory model if present)
if 'model' in globals():
    trained_model = model
else:
    model_path = "/content/trained_model.pkl"
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Trained model file not found at {model_path}. Train the model (Cell 11) or upload the file.")
    with open(model_path, "rb") as f:
        trained_model = pickle.load(f)

# --- Your mapping & required features (use the variables you provided) ---
standard_mapping = {
    # RBC Variants
    "Red blood cell count (RBC)": "RBC",
    "RBC Count": "RBC",
    "Erythrocyte Count": "RBC",
    "Red Cells": "RBC",
    "Red blood cells": "RBC",
    # Hemoglobin
    "Hemoglobin": "Hemoglobin",
    "Hb": "Hemoglobin",
    "Hb(HEMOGLOBIN)": "Hemoglobin",
    "Hemoglobin level": "Hemoglobin",
    # Hematocrit
    "Hematocrit": "Hematocrit",
    "Het": "Hematocrit",
    "HCT": "Hematocrit",
    "PCV (Packed Cell Volume)": "Hematocrit",
    # WBC
    "White blood cell count (WBC)": "WBC",
    "WBC Count": "WBC",
    "Leukocytes": "WBC",
    "Total Leukocyte Count (TLC)": "WBC",
    # Platelets
    "Platelet Count": "Platelets",
    "Platelets": "Platelets",
    "PLT": "Platelets",
    "Thrombocytes": "Platelets",
    # BP
    "Blood Pressure": ["BP_Systolic", "BP_Diastolic"],
    "BP": ["BP_Systolic", "BP_Diastolic"],
    "Systolic BP": "BP_Systolic",
    "Diastolic BP": "BP_Diastolic",
    # Other
    "Mean Corpuscular Volume (MCV)": "MCV",
    "MCV": "MCV",
    "Mean Cell Volume": "MCV",
    "Mean Corpuscular Hemoglobin (MCH)": "MCH",
    "MCH": "MCH",
    "Mean Cell Hemoglobin": "MCH",
    "Mean Corpuscular Hemoglobin Concentration (MCHC)": "MCHC",
    "MCHC": "MCHC",
    "Mean Cell Hemoglobin Concentration": "MCHC",
    "Neutrophils": "DC Neutrophils",
    "Neutrophils polynuclear": "DC Neutrophils",
    "DC Neutrophils": "DC Neutrophils",
    "DC Neutrophil Count": "DC Neutrophils",
    "Lymphocytes": "Lymphocytes",
    "Monocytes": "Monocytes",
    "Eosinophils": "Eosinophils",
    "Eosinophil Count": "Eosinophils",
    "ESR": "ESR",
    "Erythrocyte Sedimentation Rate": "ESR",
    "FBS": "FBS",
    "Fasting Blood Sugar": "FBS",
    "Fasting Glucose": "FBS",
    "LDL Cholesterol": "LDL",
    "HDL Cholesterol": "HDL",
    "Triglycerides (TG)": "TG",
    "Triglycerides": "TG",
}

required_features = [
    "Age", "Weight", "Length", "BMI", "DM", "HTN", "FAMILYHTN", "CurrentSmoker", "EXSmoker", "FH",
    "Obesity", "CHAGHISHEKAMI", "CRF", "CVA", "Airwaydisease", "ThyroidDisease", "HLP", "STRESS", "noise",
    "shiftwork", "BP_Systolic", "BP_Diastolic", "Heart Rate", "Edema", "WeakPeripheralPulse", "Lungrales",
    "SystolicMurmur", "DiastolicMurmur", "ChestPain", "Dyspnea", "heartbeat", "syanoz", "Function",
    "exercisetest", "arytmi", "FBS", "CR", "TG", "LDL", "HDL", "BUN", "RBC", "Hemoglobin", "POLY",
    "WBC", "Lymph", "eo", "Platelets", "Hematocrit"
]

# --- Standardize keys from extracted_values (case-insensitive matching) ---
pdf_values = extracted_values.copy()  # from OCR step

# Build lowercase mapping for quick matching
lower_map = {k.lower(): v for k, v in standard_mapping.items()}

mapped_values = {}
for raw_k, raw_v in pdf_values.items():
    if raw_k is None:
        continue
    k = str(raw_k).strip()
    kl = k.lower()
    if kl in lower_map:
        mapped_key = lower_map[kl]
        # If mapped_key is a list -> BP special handling
        if isinstance(mapped_key, list):
            # Expect value like "120/80" or "120 / 80 mmHg"
            bp_str = str(raw_v)
            sys_bp, dia_bp = extract_bp_parts(bp_str)
            if sys_bp is not None and dia_bp is not None:
                mapped_values["BP_Systolic"] = sys_bp
                mapped_values["BP_Diastolic"] = dia_bp
            else:
                # fallback: try splitting by non-digits
                parts = re.split(r'[^\d]+', bp_str)
                nums = [p for p in parts if p.isdigit()]
                if len(nums) >= 2:
                    mapped_values["BP_Systolic"] = float(nums[0])
                    mapped_values["BP_Diastolic"] = float(nums[1])
                else:
                    # store raw so later logic may handle
                    mapped_values["Blood Pressure"] = bp_str
        else:
            mapped_values[mapped_key] = parse_number(raw_v)
    else:
        # not in mapping: keep original key but normalized key name (strip)
        mapped_values[k] = parse_number(raw_v)

# Additional: if pdf had "BP_Systolic"/"BP_Diastolic" already as keys, keep them
for key in ("BP_Systolic","BP_Diastolic"):
    if key not in mapped_values and key in pdf_values:
        mapped_values[key] = parse_number(pdf_values[key])

# --- Build final input dict aligned to model features ---
# Get the training feature columns (ensure same order)
model_columns = list(df.drop(columns=["heartattack"]).columns)

# Prepare row dict: default 0.0
row = {col: 0.0 for col in model_columns}

# Try to fill row from mapped_values (matching exact names if possible)
for col in model_columns:
    # direct exact match
    if col in mapped_values and mapped_values[col] is not None:
        try:
            row[col] = float(mapped_values[col])
        except:
            row[col] = 0.0
        continue
    # case-insensitive key match
    for mv_key, mv_val in mapped_values.items():
        if mv_val is None:
            continue
        if mv_key.lower() == col.lower():
            try:
                row[col] = float(mv_val)
            except:
                row[col] = 0.0
            break
    # handle common synonyms (Platelet Count -> Platelets etc.)
    if row[col] == 0.0:
        if col.lower() in ("platelets","platelet count") and ("Platelets" in mapped_values or "Platelet Count" in mapped_values):
            v = mapped_values.get("Platelets", mapped_values.get("Platelet Count"))
            if v is not None:
                try:
                    row[col] = float(v)
                except:
                    row[col] = 0.0

# If Blood Pressure stored as "Blood Pressure" string, split now
if "Blood Pressure" in mapped_values and (row.get("BP_Systolic", 0.0) == 0.0 and row.get("BP_Diastolic", 0.0) == 0.0):
    sys_bp, dia_bp = extract_bp_parts(str(mapped_values["Blood Pressure"]))
    if sys_bp is not None:
        if "BP_Systolic" in row: row["BP_Systolic"] = sys_bp
    if dia_bp is not None:
        if "BP_Diastolic" in row: row["BP_Diastolic"] = dia_bp

# --- Create DataFrame aligned to training columns ---
new_input_df = pd.DataFrame([row], columns=model_columns)

# --- Scale and predict ---
new_input_scaled = scaler.transform(new_input_df)

try:
    prediction = trained_model.predict(new_input_scaled)
except Exception as e:
    raise RuntimeError(f"Model prediction failed: {e}")

# Optional: probabilities
probs = None
if hasattr(trained_model, "predict_proba"):
    try:
        probs = trained_model.predict_proba(new_input_scaled)[0].tolist()
    except Exception:
        probs = None

# --- Output ---
print("\n✅ Mapped values (sample):")
for k in sorted(list(mapped_values.keys()))[:40]:
    print(f"  {k}: {mapped_values[k]}")

print("\n✅ Prepared input (non-zero features):")
for c, v in new_input_df.T[0].items():
    if v != 0.0:
        print(f"  {c}: {v}")

print("\n🔍 Prediction Result:")
print("  Predicted label (heartattack):", int(prediction[0]))
if probs is not None:
    print("  Predicted probabilities:", probs)
print("  Interpretation:", "✅ Heart Attack Risk: Yes" if int(prediction[0]) == 1 else "❌ No Heart Attack Risk")

# --- Report missing required features (from your required_features list) ---
missing_required = [f for f in required_features if f not in mapped_values or mapped_values.get(f) is None]
if missing_required:
    print("\nNote: These required features were not found in extracted values (set to 0):")
    print(", ".join(missing_required))
else:
    print("\nAll required features were found in the extracted values.")


✅ Mapped values (sample):
  Abs. Basophil: 0.0
  Abs. Eosinophil: 0.4
  Abs. Lymphocyte: 1.2
  Abs. Monocyte: 0.7
  Abs. Neutrophil: 3.1
  Basophil %: 1.0
  Eosinophil %: 7.0
  Hematocrit: 47.2
  Hemoglobin: 16.0
  Lymphocyte %: 23.0
  MCH: 30.8
  MCHC: 33.9
  MCV: 91.0
  Monocyte %: 13.0
  Neutrophil %: 56.0
  Platelets: 149.0
  RBC: 5.2
  RDW: 12.7
  WBC: 5.4

✅ Prepared input (non-zero features):
  RBC: 5.2
  WBC: 5.4

🔍 Prediction Result:
  Predicted label (heartattack): 1
  Predicted probabilities: [0.65, 0.35]
  Interpretation: ✅ Heart Attack Risk: Yes

Note: These required features were not found in extracted values (set to 0):
Age, Weight, Length, BMI, DM, HTN, FAMILYHTN, CurrentSmoker, EXSmoker, FH, Obesity, CHAGHISHEKAMI, CRF, CVA, Airwaydisease, ThyroidDisease, HLP, STRESS, noise, shiftwork, BP_Systolic, BP_Diastolic, Heart Rate, Edema, WeakPeripheralPulse, Lungrales, SystolicMurmur, DiastolicMurmur, ChestPain, Dyspnea, heartbeat, syanoz, Function, exercisetest, arytmi, FBS

In [ ]:
!pip install google-generativeai

In [ ]:
import os
import json
import requests

# ✅ Retrieve Mistral API Key Securely
MISTRAL_API_TOKEN = os.getenv("MISTRAL_API_KEY")

def generate_medical_explanation(numeric_values):
    """Dynamically generates an AI-powered medical explanation using Mistral."""

    if not numeric_values:
        return "Error: No extracted medical values found."

    # Format extracted values into a readable format
    values_summary = "\n".join([f"{k}: {v}" for k, v in numeric_values.items()])

    # AI Prompt using extracted medical values
    prompt = f"""
    You are a medical expert. Based on the following patient's lab test results:

    {values_summary}

    Generate a structured **concise** medical report including:
    - Explanation of any abnormal conditions.
    - Causes and associated risks.
    - Recommended lifestyle changes.
    - Urgent precautions if necessary.
    - Keep it **brief but informative**.
    """

    # ✅ API Call to Mistral
    headers = {
        "Authorization": f"Bearer {MISTRAL_API_TOKEN}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "open-mixtral-8x7b",  # Best for structured output
        "messages": [
            {"role": "system", "content": "Generate a structured medical report based on the patient's lab test results."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.5
    }

    response = requests.post("https://api.mistral.ai/v1/chat/completions", headers=headers, json=payload)

    if response.status_code != 200:
        return f"Error: Mistral API failed - {response.text}"

    try:
        # Extract the AI response text
        mistral_response = response.json()["choices"][0]["message"]["content"]
        return mistral_response.strip()
    except (json.JSONDecodeError, KeyError):
        return "Error: Failed to parse Mistral response."

# ✅ Example usage with extracted values
numeric_values = extracted_values  # Replace with your actual extracted values

report= generate_medical_explanation(numeric_values)

# ✅ Print AI-generated report
print("\n📝 AI-Generated Medical Report:\n")
print(report)



📝 AI-Generated Medical Report:

Patient's Lab Test Results:

1. White Blood Cell (WBC) count is within normal limits at 5.4 (4.5-11.0 x 10^3/uL).
2. Red Blood Cell (RBC) count is slightly decreased to 5.20 (4.5-6.2 x 10^6/uL), which may indicate mild anemia.
3. Hemoglobin is at the lower end of normal with 16.0 g/dL (14.0-18.0 g/dL for men, 12.0-16.0 g/dL for women).
4. Hematocrit is slightly low at 47.2% (40.7-50.3% for men, 36.1-44.3% for women), also indicating possible mild anemia.
5. Mean Corpuscular Volume (MCV) is within normal limits at 91 fL (80-100 fL).
6. Mean Corpuscular Hemoglobin (MCH) is slightly elevated to 30.8 pg (27.0-33.0 pg).
7. Mean Corpuscular Hemoglobin Concentration (MCHC) is within normal limits at 33.9 g/dL (32.0-36.0 g/dL).
8. Red Cell Distribution Width (RDW) is slightly elevated to 12.7% (11.5-14.5%), which can be seen in various anemias.
9. Platelet count is slightly decreased to 149 x 10^3/uL (150-450 x 10^3/uL), but not low enough to be concerning.
1


In [ ]:
!pip install googletrans==3.1.0a0

  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.16.0
    Uninstalling h11-0.16.0:
      Successfully uninstalled h11-0.16.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.9
    Uninstalling httpcore-1.0.9:
      Successfully uninstalled httpcore-1.0.9
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mistralai 1.9.11 require

In [ ]:
from googletrans import Translator

def translate_report(report_text, target_language):
    """Translates the AI-generated report into the specified language."""
    translator = Translator()
    translated_text = translator.translate(report_text, dest=target_language).text
    return translated_text

# ✅ Translate into Telugu & Tamil
translated_telugu = translate_report(report, "te")
translated_tamil = translate_report(report, "ta")

# ✅ Print Translated Reports
print("\n📄 AI-Generated Report (Telugu):\n", translated_telugu)
print("\n📄 AI-Generated Report (Tamil):\n", translated_tamil)



📄 AI-Generated Report (Telugu):
 పేషెంట్స్ ల్యాబ్ టెస్ట్ ఫలితాలు:

1. తెల్ల రక్త కణం (WBC) కౌంట్ సాధారణ పరిమితుల్లో 5.4 (4.5-11.0 x 10^3/uL) వద్ద ఉంటుంది.
2. రెడ్ బ్లడ్ సెల్ (RBC) కౌంట్ 5.20 (4.5-6.2 x 10^6/uL)కి కొద్దిగా తగ్గింది, ఇది తేలికపాటి రక్తహీనతను సూచిస్తుంది.
3. హిమోగ్లోబిన్ 16.0 g/dL (పురుషులకు 14.0-18.0 g/dL, స్త్రీలకు 12.0-16.0 g/dL)తో సాధారణ స్థాయికి దిగువన ఉంది.
4. హెమటోక్రిట్ 47.2% (పురుషులకు 40.7-50.3%, స్త్రీలకు 36.1-44.3%) వద్ద కొంచెం తక్కువగా ఉంటుంది, ఇది తేలికపాటి రక్తహీనతను కూడా సూచిస్తుంది.
5. మీన్ కార్పస్కులర్ వాల్యూమ్ (MCV) 91 fL (80-100 fL) వద్ద సాధారణ పరిమితుల్లో ఉంటుంది.
6. మీన్ కార్పస్కులర్ హేమోగ్లోబిన్ (MCH) 30.8 pg (27.0-33.0 pg)కి కొద్దిగా పెరిగింది.
7. మీన్ కార్పస్కులర్ హిమోగ్లోబిన్ ఏకాగ్రత (MCHC) 33.9 g/dL (32.0-36.0 g/dL) వద్ద సాధారణ పరిమితుల్లో ఉంటుంది.
8. రెడ్ సెల్ డిస్ట్రిబ్యూషన్ వెడల్పు (RDW) కొద్దిగా 12.7% (11.5-14.5%)కి పెరిగింది, ఇది వివిధ రక్తహీనతలలో చూడవచ్చు.
9. ప్లేట్‌లెట్ కౌంట్ 149 x 10^3/uL (150-450 x 10^3/uL)కి కొద్దిగా తగ్గింది, కానీ ఆం

In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=4b90e3e94612bae2d90c1c8fb6384e7ccdb91f837b9936aa7b63ef0afef8749d
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [ ]:
from fpdf import FPDF

def save_report_as_pdf(report_text, filename="Medical_Report.pdf"):
    """Saves the AI-generated medical report as a PDF file."""
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Add title
    pdf.set_font("Arial", style='B', size=16)
    pdf.cell(200, 10, "Medical Report", ln=True, align='C')
    pdf.ln(10)

    # Add content, encoding the text with 'utf-8'
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, report_text.encode('utf-8').decode('latin-1','ignore')) # Encode with utf-8 and decode ignoring errors

    # Save the PDF
    pdf.output(filename)
    print(f"✅ Report saved as {filename}")

# ✅ Save reports as PDF
save_report_as_pdf(report, "Medical_Report_English.pdf")
save_report_as_pdf(translated_telugu, "Medical_Report_Telugu.pdf")
save_report_as_pdf(translated_tamil, "Medical_Report_Tamil.pdf")

✅ Report saved as Medical_Report_English.pdf
✅ Report saved as Medical_Report_Telugu.pdf
✅ Report saved as Medical_Report_Tamil.pdf
